In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
# Before reading in the csv file, I removed commas from the Area_m2 column and renamed %_Cover to pct_cover
data_dir = 'D:/crs/proj/2023_NCB_recovery/'
df = pd.read_csv(data_dir+'Over_and_Sherwood_2024_VegetationCoverRaw.csv')
df = df.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12]]
# Remove ROIs
df = df[df['Type'] != 'ROI']
df

,ID,Area_m2,Date,Location,Hurricane,Type,Dense_Vegetation,Sparse_Vegetation,Vegetation Total,Sand,Water,pct_Cover
3,0.0,3045.00,20190830,NCB,Dorian,Washout,1223.0,4281.0,5504.0,6738,0,44.96
4,1.0,13697.00,20190830,NCB,Dorian,Washout,1709.0,24125.0,25834.0,29304,0,46.85
5,2.0,7891.00,20190830,NCB,Dorian,Washout,1290.0,16588.0,17878.0,13966,0,56.14
6,3.0,2646.00,20190830,NCB,Dorian,Washout,896.0,4909.0,5805.0,4936,0,54.05
7,4.0,7157.00,20190830,NCB,Dorian,Washout,1717.0,10235.0,11952.0,16952,0,41.35
...,...,...,...,...,...,...,...,...,...,...,...,...
592,5.0,13790.50,20210404,FI,Sandy,Washover,NaN,NaN,121675.0,98999,0,55.24
593,6.0,7025.97,20210404,FI,Sandy,Washover,NaN,NaN,56792.0,55626,0,50.20
594,7.0,5939.14,20210404,FI,Sandy,Washover,NaN,NaN,49922.0,45088,0,57.61
595,8.0,13478.30,20210404,FI,Sandy,Washover,NaN,NaN,136340.0,79289,0,57.33


In [34]:
# I used these to check for weird values and found commas and quotes in the Area_m2 column, which made them read in as objects.
print(df[['Area_m2', 'pct_Cover']].dtypes)  # Ensure both columns are numeric
print(df[['Area_m2', 'pct_Cover']].isnull().sum())  # Check for missing values

Area_m2      float64
pct_Cover    float64
dtype: object
Area_m2      0
pct_Cover    0
dtype: int64


In [35]:
df['veg_area_m2'] = df['Area_m2']*df['pct_Cover']/100 

In [36]:
# Find the unique combinations of 'Date', 'Location', 'Hurricane', 'Type'
unique_combos = df[['Date', 'Location', 'Hurricane', 'Type']].drop_duplicates()
print(unique_combos)

         Date Location Hurricane      Type
3    20190830      NCB    Dorian   Washout
86   20191011      NCB    Dorian   Washout
169  20200928      NCB    Dorian   Washout
252  20221020      NCB    Dorian   Washout
337  20181007      NCB  Florence  Washover
426  20190830      NCB  Florence  Washover
515  20170325      SJI    Harvey   Washout
524  20170829      SJI    Harvey   Washout
533  20200131      SJI    Harvey   Washout
542  20220323      SJI    Harvey   Washout
551  20230625      SJI    Harvey   Washout
560  20080519       BP       Ike   Washout
562  20100108       BP       Ike   Washout
564  20130419       BP       Ike   Washout
566  20180102       BP       Ike   Washout
568  20220121       BP       Ike   Washout
570  20171001       FI     Sandy  Washover
579  20190919       FI     Sandy  Washover
588  20210404       FI     Sandy  Washover


In [39]:
# Loop through each unique combination and make a dataframe
for index, row in unique_combos.iterrows():
    # Filter the original DataFrame for the current unique combination
    fil_df = df[
        (df['Date'] == row['Date']) &
        (df['Location'] == row['Location']) &
        (df['Hurricane']== row['Hurricane']) &
        (df['Type'] == row['Type'])
    ]
    # Mean veg. percentage
    vmean = fil_df['pct_Cover'].mean()
    area_tot = fil_df['Area_m2'].sum()
    varea_tot = fil_df['veg_area_m2'].sum()
    vamean = 100.*varea_tot/area_tot
    df_name = f"{row['Date']}_{row['Location']}_{row['Hurricane']}_{row['Type']}"
    df_date = row['Date']
    print('{}, {:.0f}, {:.2f}, {:.2f}, {:.0f}'.format( df_name, df_date, vmean, vamean, area_tot ))



20190830_NCB_Dorian_Washout 20190830, 44.09, 55.17, 1094970
20191011_NCB_Dorian_Washout 20191011, 14.72, 12.81, 1094970
20200928_NCB_Dorian_Washout 20200928, 12.70, 13.16, 1094970
20221020_NCB_Dorian_Washout 20221020, 20.12, 23.58, 1094970
20181007_NCB_Florence_Washover 20181007, 2.93, 3.99, 929929
20190830_NCB_Florence_Washover 20190830, 37.58, 38.42, 929929
20170325_SJI_Harvey_Washout 20170325, 74.23, 74.97, 513039
20170829_SJI_Harvey_Washout 20170829, 2.25, 3.36, 513039
20200131_SJI_Harvey_Washout 20200131, 7.81, 8.03, 513039
20220323_SJI_Harvey_Washout 20220323, 4.15, 3.51, 513039
20230625_SJI_Harvey_Washout 20230625, 6.39, 5.74, 513039
20080519_BP_Ike_Washout 20080519, 81.12, 81.12, 36724
20100108_BP_Ike_Washout 20100108, 13.53, 13.53, 36724
20130419_BP_Ike_Washout 20130419, 44.20, 44.20, 36724
20180102_BP_Ike_Washout 20180102, 39.84, 39.84, 36724
20220121_BP_Ike_Washout 20220121, 14.92, 14.93, 36724
20171001_FI_Sandy_Washover 20171001, 51.44, 52.97, 124402
20190919_FI_Sandy_Washo

In [ ]:
plt.plot('df['